In [1]:
import mne
import os
import scipy.io
import listen_italian_functions
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')
from tqdm import tnrange, tqdm_notebook
from itertools import combinations

data_path = os.path.dirname(os.path.dirname(os.getcwd()))

subject_name = ['Alice','Andrea','Daniel','Elena','Elenora','Elisa','Federica','Francesca','Gianluca1','Giada','Giorgia',
                'Jonluca','Laura','Leonardo','Linda','Lucrezia','Manu','Marco','Martina','Pagani','Pasquale','Sara',
                'Silvia','Silvia2','Tommaso']

remove_first = 0.5 #seconds


In [ ]:
Tmin = 0
Tmax = 3.51
trial_len = 2

GA_epoches = []
for s in subject_name:
    save_path = data_path + '/python/data/coherence_epochs/'+s+'-coh-epo-'+str(Tmin)+'-' \
    +str(Tmax)+'-trialLen-'+str(trial_len)+'.fif'
    epochs = mne.read_epochs(save_path)
    GA_epoches.append(epochs)
    print('----------------------------------------------------------------------------------------------------------------'+s)


This filename (D:\projects\current\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Alice-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading D:\projects\current\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Alice-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
124 matching events found
No baseline correction applied
124 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Alice
This filename (D:\projects\current\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Andrea-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs

This filename (D:\projects\current\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Pasquale-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif or -epo.fif.gz
Reading D:\projects\current\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Pasquale-coh-epo-0-3.51-trialLen-2.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    3507.50 ms
        0 CTF compensation matrices available
117 matching events found
No baseline correction applied
117 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
----------------------------------------------------------------------------------------------------------------Pasquale
This filename (D:\projects\current\listen_italian_motor_entrainment\analysis/python/data/coherence_epochs/Sara-coh-epo-0-3.51-trialLen-2.fif) does not conform to MNE naming conventions. All

In [ ]:
iter_freqs = [
    ('fr', 1, 3),
    #('fr', 2, 4),
    #('fr', 3, 5),
    ('fr', 4, 6)
    #('fr', 5, 7),
    #('fr', 6, 8),
    #('fr', 7, 9),
    #('fr', 8, 10),
    #('fr', 9, 11),
    #('fr', 10, 12)
]

save_path = data_path +'/python/data/partialCoh/partialCoh-trailLen-' +str(trial_len)+'-removedFirst-'+ str(remove_first)

features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
condition = ['Hyper','Normal','Hypo','All']
delay = np.arange(0,1.1,0.1)


extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'Hyper': 1,'Normal': 2,'Hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
        
for s in range(0,len(subject_name)):
    for d in delay:   
        for c in condition:
            for fr in range(0,len(iter_freqs)):
                b = str(iter_freqs[fr][0])+ '-'+str(iter_freqs[fr][1])+'-'+str(iter_freqs[fr][2])+'Hz'

                fmin = iter_freqs[fr][1]
                fmax = iter_freqs[fr][2]
                a = listen_italian_functions.partialCoherence_preprocess_delay(GA_epoches[s],remove_first,d,
                                                                               trial_len,extra_channels,eeg_channles,
                                                                               info,ch_names,event_id,fmin,fmax,c)   
    
                d = d.round(decimals=1)
                scipy.io.savemat(save_path+'s-'+'condition-'+c+'-delay-'+str(d)+'-'+b+'-' +subject_name[s],a)
        
        

124 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
    using multitaper spectrum estimation with 7 DPSS windows
Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
    using multitaper spectrum estimation with 7 DPSS windows
Computing cross-spectral density from epochs...
    Computing CSD matrix for epoch 1
    Computing CSD matrix for epoch 2
    Computing CSD matrix for epoch 3
    Computing CSD matrix for epoch 4
    Computing CSD matrix for epoch 5
    Computing CSD matrix for epoch 6
    Computing CSD matrix for epoch 7
    Computing CSD matrix for epoch 8
    Computing CSD matrix for epoch 9
    Computing CSD matrix for epoch 10
    Computing CSD matrix for epoch 11
    Computing CSD matrix for epoch 12
    Computing CSD matrix for epoch 13
    Computing CSD matrix for epoch 14
    Computing CSD matrix for epoch 15
    Computing CSD matrix for epoch 1

In [ ]:
con, freqs, times, n_epochs, n_tapers = spectral_connectivity(epochs, method='coh',mode='multitaper', sfreq=sfreq, 
                                                              fmin=fmin, fmax=fmax,indices=indices,faverage=True, 
                                                              tmin=0, mt_adaptive=False, block_size=1000,verbose='ERROR')



partial_coh_XY_R = (abs(conXY - conXR * conRY) **2) / ((1 - abs(conXR)**2) * (1 - abs(conRY)**2))



# save epoches for matlab filetrip partial coherence

In [ ]:
remove_first = 0.5#seconds



save_path = data_path +'/data/partialCoh/partialCoh-trailLen-' +str(trial_len)+'-removedFirst-'+ str(remove_first)

features = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
condition = ['Hyper','Normal','Hypo']
delay = np.arange(0,1.1,0.1)


extra_channels = ['envelop','jawaopening','lipaparature','lipProtrusion','TTCD','TMCD','TBCD']
eeg_channles = np.setdiff1d(GA_epoches[0].ch_names, extra_channels)
event_id = {'Hyper': 1,'Normal': 2,'Hypo': 3}
ch_types = np.repeat('eeg', len(features)+59)
ch_names = np.hstack((eeg_channles,features))        
info = mne.create_info(ch_names = ch_names.tolist(),ch_types = ch_types,sfreq = GA_epoches[0].info['sfreq'])
ch_names = np.setdiff1d(extra_channels,features)
        
for s in tqdm_notebook(range(0,len(subject_name)), desc='Subjects'):
    frames = []
    for d in tqdm_notebook(delay, desc='Delay'):        

        epoch = listen_italian_functions.coherence_preprocess_delay(GA_epoches[s],remove_first,d,trial_len,
                                                                    extra_channels,eeg_channles,info,ch_names,event_id)

         
        d = d.round(decimals=1)
        for c in condition:
            a = epoch[c]
            a.save(save_path+'s-'+'condition-'+c+'-delay-'+str(d)+'s-' +subject_name[s]+'.fif')

        epoch.save(save_path+'s-'+'condition-All-delay-'+str(d)+'s-' +subject_name[s]+'.fif')    
            
            
           